In [151]:
## Create constants and import hash libraries
# Prime order of curve
P = 2**256 - 2**32 - 977
# The order of the base point (also known as the generator point G),
# which is the number of distinct points on the curve that can be generated by
# repeatedly adding the base point to itself.
Q = 0xfffffffffffffffffffffffffffffffebaaedce6af48a03bbfd25e8cd0364141
# Generator point Cordinates
G_x = 0x79be667ef9dcbbac55a06295ce870b07029bfcdb2dce28d959f2815b16f81798
G_y = 0x483ada7726a3c4655da4fbfc0e1108a8fd17b448a68554199c47d08ffb10d4b8
# Protect against replay attacks
CONTEXT = b'FROST-BIP340'


In [152]:
# Define ECC point class
class Point:
        """Class representing an elliptic curve point."""
        def __init__(self, x=float('inf'), y=float('inf')):
            self.x = x
            self.y = y

        @classmethod
        def secret_deserialize(self, hex_public_key):
            hex_bytes = bytes.fromhex(hex_public_key)
            is_even = hex_bytes[0] == 2
            x_bytes = hex_bytes[1:]
            x = int.from_bytes(x_bytes, 'big')
            y_squared = (pow(x, 3, P) + 7) % P
            y = pow(y_squared, (P + 1) // 4, P)

            if y % 2 == 0:
                even_y = y
                odd_y = (P - y) % P
            else:
                even_y = (P - y) % P
                odd_y = y
            y = even_y if is_even else odd_y

            return self(x, y)

        def secret_serialize(self):
            # Return compressed key
            prefix = b'\x02' if self.y % 2 == 0 else b'\x03'

            return prefix + self.x.to_bytes(32, 'big')

        @classmethod
        def xonly_deserialize(self, hex_public_key):
            hex_bytes = bytes.fromhex(hex_public_key)
            x = int.from_bytes(hex_bytes, 'big')
            y_squared = (pow(x, 3, P) + 7) % P
            y = pow(y_squared, (P + 1) // 4, P)

            if y % 2 != 0:
                y = (P - y) % P

            return self(x, y)

        def xonly_serialize(self):
            return self.x.to_bytes(32, 'big')

        # point at infinity
        def is_zero(self):
            return self.x == float('inf') or self.y == float('inf')

        def __eq__(self, other):
            return self.x == other.x and self.y == other.y

        def __ne__(self, other):
            return not self == other

        def __neg__(self):
            if self.is_zero():
                return self

            return self.__class__(self.x, P - self.y)

        # Double point
        def dbl(self):
            x = self.x
            y = self.y
            s = (3 * x * x * pow(2 * y, P - 2, P)) % P
            sum_x = (s * s - 2 * x) % P
            sum_y = (s * (x - sum_x) - y) % P

            return self.__class__(sum_x, sum_y)

        def __add__(self, other):
            if self == other:
                return self.dbl()
            if self.is_zero():
                return other
            if other.is_zero():
                return self
            if self.x == other.x and self.y != other.y:
                return self.__class__()
            s = ((other.y - self.y) * pow(other.x - self.x, P - 2, P)) % P
            sum_x = (s * s - self.x - other.x) % P
            sum_y = (s * (self.x - sum_x) - self.y) % P

            return self.__class__(sum_x, sum_y)

        def __rmul__(self, scalar):
            p = self
            r = self.__class__()
            i = 1

            while i <= scalar:
                if i & scalar:
                    r = r + p
                p = p.dbl()
                i <<= 1

            return r

        def __str__(self):
            if self.is_zero():
                return '0'
            return 'X: 0x{:x}\nY: 0x{:x}'.format(self.x, self.y)

        def __repr__(self) -> str:
            return self.__str__()

In [153]:
# Create ECC Generator point
G = Point(G_x, G_y)


## DKG
Perdensen key generation is a two step process
1. Generate and distribute public shares, polynomial coefficients, and commitments to the shares
2. Participants sign for their commitment other participants verify


TODO why is this important


In [154]:
import secrets
from hashlib import sha256

# Lets create a 2:3 multisig
THRESHOLD = 2
N = 3

def new_polynomial(threshold):
    # 1. Generate Shamir polynomial with random coefficients, and with degree
    # equal to the threshold minus one.
    coefficients = [secrets.randbits(256) % Q for _ in range(threshold)]
    return coefficients


In [155]:
# Participant needs to prove ownership over first coef
# And commit to their index in the signing session
# This is done by signing a nonce and their index
def proof_of_knowledge(index, coefficients):
    # Use a new nonce to avoid signature replay attack
    # k ⭠ ℤ_q
    nonce = secrets.randbits(256) % Q
    # R = g^k
    nonce_point = nonce * G
    # a_0 (first coef) is always the partial private key
    secret = coefficients[0]
    secret_commitment = secret * G

    # c = H(i, 𝚽, s, R)
    # Commit to the your position in signing session
    # The frost tag
    # Your partial secret key
    # Your public partial nonce
    challenge_input = index.to_bytes(1, 'big') + CONTEXT + secret_commitment.secret_serialize() + nonce_point.secret_serialize()
    challenge_hash = sha256(challenge_input)
    challenge_hash_bytes = challenge_hash.digest()
    challenge_hash_int = int.from_bytes(challenge_hash_bytes, 'big')
    # s_i = k + f(0) * c
    s = (nonce + secret * challenge_hash_int) % Q

    return [nonce_point, s]


In [156]:
# We want to verify a participant verifies knowledge of their coeffcients
# Proof is a normal schnorr signature [s,r]
def verify_proof_of_knowledge(index, proof, secret_coeffecient):
    [nonce_point, s] = proof
    # c_l = H(l, 𝚽, g^a_l_0, R_l)
    # Recalculate challenge
    challenge_input = index.to_bytes(1, 'big') + CONTEXT + secret_coeffecient.secret_serialize() + nonce_point.secret_serialize()
    challenge_hash_bytes = sha256(challenge_input).digest()
    challenge_hash_int = int.from_bytes(challenge_hash_bytes, 'big')

    return nonce_point == (s * G) + (Q - challenge_hash_int) * secret_coeffecient
    

In [157]:
# Lets put it all together
# All participants share coeffecients
# The only value relevant for producing a public key is the commitment derived from the first coefficient
N = 3
THRESHOLD = 2
p1 = new_polynomial(THRESHOLD)
p2 = new_polynomial(THRESHOLD)
p3 = new_polynomial(THRESHOLD)

p1_commitments = [coefficient * G for coefficient in p1]
p2_commitments = [coefficient * G for coefficient in p2]
p3_commitments = [coefficient * G for coefficient in p3]

p1_proof = proof_of_knowledge(1, p1)
p2_proof = proof_of_knowledge(2, p2)
p3_proof = proof_of_knowledge(3, p3)




In [158]:
# Verify proofs
# Each participant needs to do this for each proof provided
assert(verify_proof_of_knowledge(1, p1_proof, p1_commitments[0]) == True)
assert(verify_proof_of_knowledge(2, p2_proof, p2_commitments[0]) == True)
assert(verify_proof_of_knowledge(3, p3_proof, p3_commitments[0]) == True)

assert(not verify_proof_of_knowledge(2, p1_proof, p1_commitments[0]))
assert(not verify_proof_of_knowledge(1, p1_proof, p2_commitments[0]))


## DKG
Perdensen key generation is a two step process
1. Generate and distribute public shares, polynomial coefficients, and commitments to the shares
2. Participants sign for their commitment other participants verify


TODO why is this important


In [159]:
# Lets generate aggregate shares
# Coefs here are ints, not ecc points
def evaluate_polynomial(index, participant_coefficients):
    # f_i(x) = ∑ a_i_j * x^j, 0 ≤ j ≤ t - 1
    # Horner's method
    y = 0
    for i in range(len(participant_coefficients) - 1, -1, -1):
        y = y * index + participant_coefficients[i]

    # eval f(i) 
    return y % Q


def generate_share(n, participant_coefficients):
    # Evaluate a participants poly at each index
    shares = [evaluate_polynomial(i, participant_coefficients) for i in range(1, n + 1)]
    return shares


In [160]:

p1_shares = generate_share(N, p1)
p2_shares = generate_share(N, p2)
p3_shares = generate_share(N, p3)

In [161]:
# Extra step: lets verify each share was calculated correctly
def verify_share(shares, coefficient_commitments, index):
    # ∏ 𝜙_l_k^i^k mod q, 0 ≤ k ≤ t - 1
    expected_y_commitment = Point()
    for coef_index, coef in enumerate(coefficient_commitments):
        expected_y_commitment = expected_y_commitment + \
                    ((index ** coef_index % Q) * coef)
    # g^f_l(i) ≟ ∏ 𝜙_l_k^i^k mod q, 0 ≤ k ≤ t - 1
    return shares * G == expected_y_commitment

In [162]:
# Is our aggregated share for participant #1 correct
verify_share(p1_shares[0], p1_commitments, 1)
verify_share(p1_shares[1], p2_commitments, 1)

False

In [163]:
def aggregate_shares(participant_index, participant_share, shares):
    aggregate_share = participant_share
    for share in shares:
        aggregate_share = aggregate_share + share
    return aggregate_share % Q

In [164]:

p1_agg = aggregate_shares(1, p1_shares[0],[p2_shares[0], p3_shares[0]])
p2_agg = aggregate_shares(2, p2_shares[1],[p1_shares[1], p3_shares[1]])
p3_agg = aggregate_shares(3, p3_shares[2],[p2_shares[2], p1_shares[2]])


In [165]:
# x_0 for each participant polynomials
def derive_public_key(coefficient_commitments):
    public_key = coefficient_commitments[0]
    for commitment in coefficient_commitments[1:]:
        public_key = public_key + commitment

    return public_key



In [166]:
aggregate_public_key = derive_public_key([p1_commitments[0], p2_commitments[0], p3_commitments[0]])
aggregate_public_key


X: 0xfe70cedd51dad86092a83903f2f9dcafbb1e10d9b44cdf3a4ce247b56d3b00a1
Y: 0xb9b4c2eb00436fc8007efb0c4c78f488ac4b7eef956c5cd755b14a5057b34888

In [167]:
 def lagrange_coefficient(participant_indexes, my_index):
    numerator = 1
    denominator = 1
    for index in participant_indexes:
        if index == my_index:
            continue
        numerator = numerator * index
        denominator = denominator * (index - my_index)
    return (numerator * pow(denominator, Q - 2, Q)) % Q

In [168]:
# Lets reconstruct the secret


l1 = lagrange_coefficient([2], 1)
l2 = lagrange_coefficient([1], 2)
l3 = lagrange_coefficient([2], 3)


secret = ((p1_agg * l1) + (p2_agg * l2)) % Q
assert(secret * G == aggregate_public_key)


In [169]:
# Onto signing


In [170]:
def generate_nonces():
    # Preprocess(π) ⭢  (i, ⟨(D_i_j, E_i_j)⟩), 1 ≤ j ≤ π

    # (d_i_j, e_i_j) ⭠ $ ℤ*_q x ℤ*_q
    nonce_pair = [secrets.randbits(256) % Q, secrets.randbits(256) % Q]
    return nonce_pair


In [171]:
p1_nonce_pair = generate_nonces()
p2_nonce_pair = generate_nonces()
p3_nonce_pair = generate_nonces()

p1_nonce_point = [p1_nonce_pair[0] * G, p1_nonce_pair[1] * G]
p2_nonce_point = [p2_nonce_pair[0] * G, p2_nonce_pair[1] * G]
p3_nonce_point = [p3_nonce_pair[0] * G, p3_nonce_pair[1] * G]


In [172]:


def binding_value(index, message, nonce_commitment_pairs, participant_indexes):
    bv = sha256()
    # B
    nonce_commitment_pairs_bytes = []
    for index in participant_indexes:
        participant_pair = nonce_commitment_pairs[index-1]
        participant_pair_bytes = b''.join([commitment.secret_serialize() for commitment in participant_pair])
        nonce_commitment_pairs_bytes.append(participant_pair_bytes)
    nonce_commitment_pairs_bytes = b''.join(nonce_commitment_pairs_bytes)
    # p_l = H_1(l, m, B), l ∈ S
    pre_image = index.to_bytes(1, 'big') + message + nonce_commitment_pairs_bytes
    bv = sha256(pre_image)
    binding_value_bytes = bv.digest()

    return int.from_bytes(binding_value_bytes, 'big')

def group_commitment(message, nonce_commitment_pairs, participant_indexes):
    gc = Point()
    for index in participant_indexes:
        # p_l = H_1(l, m, B), l ∈ S
        bv = binding_value(index, message, nonce_commitment_pairs, participant_indexes)
        # D_l
        first_commitment = nonce_commitment_pairs[index-1][0]
        # E_l
        second_commitment = nonce_commitment_pairs[index-1][1]
        # R = ∏ D_l * (E_l)^p_l, l ∈ S
        gc = gc + (first_commitment + (bv * second_commitment))
    return gc

def challenge_hash(nonce_commitment, aggregate_public_key, message):
    # c = H_2(R, Y, m)
    tag_hash = sha256(b'BIP0340/challenge').digest()
    challenge_hash = sha256()
    challenge_hash.update(tag_hash)
    challenge_hash.update(tag_hash)
    challenge_hash.update(nonce_commitment.xonly_serialize())
    challenge_hash.update(aggregate_public_key.xonly_serialize())
    challenge_hash.update(message)
    challenge_hash_bytes = challenge_hash.digest()

    return int.from_bytes(challenge_hash_bytes, 'big') % Q

def sign(message, nonce_commitment_pairs, participant_indexes, signing_nonce_pair, signer_index, aggregate_public_key, aggregate_share):
    # R
    # TODO goup cmmitment 
    gc = group_commitment(message, nonce_commitment_pairs, participant_indexes)
    # c = H_2(R, Y, m)
    challenge = challenge_hash(gc, aggregate_public_key, message)
    [first_nonce, second_nonce] = signing_nonce_pair
    # Negate d_i and e_i if R is odd
    if gc.y % 2 != 0:
        first_nonce = Q - first_nonce
        second_nonce = Q - second_nonce
    # p_i = H_1(i, m, B), i ∈ S
    bv = binding_value(signer_index, message, nonce_commitment_pairs, participant_indexes)
    # λ_i
    lagrange = lagrange_coefficient(participant_indexes, my_index=signer_index)
    # Negate s_i if Y is odd
    if aggregate_public_key.y % 2 != 0:
        aggregate_share = Q - aggregate_share

    # z_i = d_i + (e_i * p_i) + λ_i * s_i * c
    return (first_nonce + (second_nonce * bv) + lagrange * aggregate_share * challenge) % Q

In [173]:
# Lets sign with the first and second participants
msg = b'fnord!'
participant_indexes = [1, 2]
nonce_commitment_pairs = [p1_nonce_point, p2_nonce_point]

s1 = sign(
    message=msg,
    nonce_commitment_pairs=nonce_commitment_pairs,
    participant_indexes=participant_indexes,
    signing_nonce_pair=p1_nonce_pair,
    signer_index=1,
    aggregate_public_key=aggregate_public_key,
    aggregate_share=p1_agg
)

s2 = sign(
    message=msg,
    nonce_commitment_pairs=nonce_commitment_pairs,
    participant_indexes=participant_indexes,
    signing_nonce_pair=p2_nonce_pair,
    signer_index=2,
    aggregate_public_key=aggregate_public_key,
    aggregate_share=p2_agg
)


In [174]:

# Lets aggregate the signature shares
def aggregate_signatures(signature_shares, gc, challenge):
    # TODO: verify each signature share
    # σ = (R, z)
    nonce_commitment = gc.xonly_serialize()
    z = (sum(signature_shares) % Q).to_bytes(32, 'big')

    return bytes.fromhex((nonce_commitment + z).hex())

In [175]:
gc = group_commitment(msg, nonce_commitment_pairs, participant_indexes)
challenge = challenge_hash(gc, aggregate_public_key, msg)
agg_sig = aggregate_signatures([s1, s2], gc=gc, challenge=challenge)
nonce_commitment = Point.xonly_deserialize(agg_sig[0:32].hex())
z = int.from_bytes(agg_sig[32:64], 'big')

# verify
# Negate Y if Y.y is odd
if aggregate_public_key.y % 2 != 0:
    aggregate_public_key = -aggregate_public_key

# R ≟ g^z * Y^-c
assert(nonce_commitment == (z * G) +(Q - challenge) * aggregate_public_key)

